# SpareRoom


In [17]:
!pip install folium

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import folium

<h1> London

<h2> Web Scraping housing info in London

In [ ]:
def get_housings(url): 
    housings_list = list()
    import requests
    from bs4 import BeautifulSoup
    # Request for home page of NY housing info
    try:
        results_page = requests.get(url)
    except:
        return None
    try:
        soup = BeautifulSoup(results_page.content,'lxml')
    except:
        return None
    # Get all the feature housings and bold housings on the first page
    feature_housings = soup.find_all('article',class_='panel-listing-result listing-featured ')
    for housing in feature_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
            housings_list.append([detail_dict,link])
    # Second Type of housing
    all_housings = soup.find_all('article', class_='panel-listing-result listing-bold ')
    for housing in all_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
            housings_list.append([detail_dict,link])
    # Third type of housing:
    early_bird_housings = soup.find_all('article', class_='panel-listing-result listing-free ')
    for housing in early_bird_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            detail_dict['Short_description'] = name
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
            housings_list.append([detail_dict,link])
#     Turn to next page and get more housings
    if soup.find('ul',class_='navnext'):
        if soup.find('ul',class_='navnext').find('a'):
            next_page_token = soup.find('ul',class_='navnext').find('a').get('href')
            next_page = 'https://www.spareroom.co.uk/flatshare/' + next_page_token
        # Use recursion to get all the attractions    
            housings_list.extend(get_housings(next_page))
    return housings_list


In [ ]:
url_london = 'https://www.spareroom.co.uk/flatshare/?search_id=745817425&'
list_ = get_housings(url_london)
print(f'We have find {len(list_)} results for you.')


In [10]:
def get_details(list_housings):
    count = 0 
    full_dict = dict()
    for dict_info, link in list_housings:
        import requests
        from bs4 import BeautifulSoup
        # Request for details of each housing in NY
        try:
            results_page = requests.get(link)
        except:
            return None
        try:
            soup = BeautifulSoup(results_page.content,'lxml')
        except:
            return None
        # Get housing price for each room 
        price_list = list()
        if soup.find('ul', class_='room-list'): 
            prices = soup.find('ul', class_='room-list')
            price = prices.find_all('strong', class_='room-list__price')
            for price_i in price:
                price_list.append(price_i.get_text())
        # price 这里似乎有个bug，但是我暂时懒得改
        if soup.find('h3', class_='feature__heading'):
            price_list.append(soup.find('h3', class_='feature__heading').get_text())
        # Clean the data to remove 'Availability' in the price list
        if 'Availability' in price_list:
            price_list.remove('Availability')
        dict_info['Price'] = price_list
        # Get Availability for each housing
        avail = soup.find('section', class_='feature feature--availability')
        if avail:
            all_info_avail = avail.find_all('dd', class_='feature-list__value')
            availablility = all_info_avail[0].get_text()      
            min_period = all_info_avail[1].get_text()      
            max_period = all_info_avail[2].get_text() 
            dict_info['Availablility'] = availablility
            dict_info['Min_period'] = min_period
            dict_info['Max_period'] = max_period
        # Get Amenities info for each housing
        amenities = soup.find('section', class_='feature feature--amenities')
        if amenities:
            all_info_amenities = amenities.find_all('dd', class_='feature-list__value')
            if all_info_amenities:
                furnish = all_info_amenities[0].get_text()
                dict_info['Furnishing'] = furnish
        else:
            dict_info['Furnishing'] = 'No info'
        # Female/Male requirement
        preference = soup.find('section', class_='feature feature--household-preferences')
        if preference:
            preference_info = preference.find_all('dt', class_='feature-list__key')
            preference_detail = preference.find_all('dd', class_='feature-list__value')
            if preference_info and preference_detail:
                for info in preference_info:
                    if info.get_text() == 'Gender':
                        info_index = preference_info.index(info)
                        break
                try:
                    gender = preference_detail[info_index].get_text() 
                    dict_info['Gender_requirement'] = gender
                except:
                    dict_info['Gender_requirement'] = 'No info'
        full_dict[count] = dict_info
        count = count + 1
    return full_dict
        

In [1]:
list_housings = list_
dict_ = get_details(list_housings)

<h3> Clean the data

In [ ]:
for key in dict_.keys():
    for subkey in dict_[key].keys():
        if isinstance(dict_[key][subkey], str):
            dict_[key][subkey] = dict_[key][subkey].lstrip().rstrip()
# Clean the price list
for key in dict_.keys():
    list_in = list()
    for item in list(dict_[key]['Price']):
        list_in.append(item.strip('\n'))
    dict_[key]['Price'] = list_in

<h3> Create a dataframe to store the info

In [ ]:
df_info = pd.DataFrame(dict_)
df_info = df_info.T

# Fix furnished problem
for i in df_info.index:
    if df_info.iloc[i]['Furnishing'] in ['Furnished', 'Part Furnished', 'Unfurnished']:
        pass
    else:
        df_info.iloc[i]['Furnishing'] = 'No info'
# Fix Gender_requirement problem
for i in df_info.index:
    if not df_info.iloc[i]['Gender_requirement'] in ['Males or females', 'Female preferred', 'No info',
       'Male preferred']:
        df_info.iloc[i]['Gender_requirement'] = 'No info'

In [ ]:
# Create a new col in df_info to store average price for a single person
pattern = r'\d+'  
price_per_month = list()

for i in df_info.index:
    list_in_price = list()
    for item in df_info.iloc[i]['Price']:
        item = item.replace(',', '')
        if 'pw' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info*4)
        if 'pcm' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info)
    price_per_month.append(list_in_price)
price_per_month = np.array(price_per_month)
df_info['Price_per_month'] = price_per_month
list_avg_price = list()
for i in df_info.index:
    list_avg_price.append(np.mean(df_info.iloc[i]['Price_per_month']) )
df_info['Average_price_per_month'] = np.array(list_avg_price)
# Create a new col in df_info to store zip code for each housing


In [ ]:
# Create a new col for zip code
list_pos = list()
for i in df_info.index:
    for s in df_info.iloc[i]['Location']:
        if s == '(':
            position = df_info.iloc[i]['Location'].index(s)
            break
    list_pos.append(df_info.iloc[i]['Location'][position:].strip('(').strip(')'))
df_info['Postal_code'] = np.array(list_pos)

<h3> Data Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')# seaborn is a style that have different colors

<h4> To see if there is a gender preference in London

In [ ]:
gender_group = df_info.groupby('Gender_requirement')
gender_group.size().plot(kind='bar', title='Gender requirement for rent in London')

<h4> To see the housing condition in postal code area in London

In [ ]:
Postal_code_group = df_info.groupby('Postal_code')
Postal_code_group.size().plot(kind='bar', title='Postal code area in London', figsize=(24,24))

<h4> To see if there is more furnished apartment in London

In [ ]:
furnish_group = df_info.groupby('Furnishing')
furnish_group.size().plot(kind='bar', title='Furnishing status of housing in London')

<h4> To see the size distribution in London

In [ ]:
size_group = df_info.groupby('Size')
size_group.size().plot(kind='bar', title='Housing size in London')

<h1> Liverpool

In [12]:
def get_housings(url): 
    housings_list = list()
    import requests
    from bs4 import BeautifulSoup
    # Request for home page of NY housing info
    try:
        results_page = requests.get(url)
    except:
        return None
    try:
        soup = BeautifulSoup(results_page.content,'lxml')
    except:
        return None
    # Get all the feature housings and bold housings on the first page
    feature_housings = soup.find_all('article',class_='panel-listing-result listing-featured ')
    for housing in feature_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    # Second type
    all_housings = soup.find_all('article', class_='panel-listing-result listing-bold ')
    for housing in all_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            detail_dict['Short_description'] = name
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
    # Third type
    early_bird_housings = soup.find_all('article', class_='panel-listing-result listing-free ')
    for housing in early_bird_housings:
        if housing.find('a'):
            detail_dict = dict()
            name = housing.find('h1').get_text()
            size = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").get_text()
            detail_dict['Short_description'] = name
            location = housing.find('div', class_='pricingInfo').find('em',class_="shortDescription").find('span', class_='listingLocation').get_text()
            # Split the room size and the location and store only size
            for i in range(1,len(size)):
                if size[i].isupper(): 
                    position = i
                    break
            detail_dict['Size'] = size[:position]
            detail_dict['Location'] = location
            link = 'https://www.spareroom.co.uk' + housing.find('a').get('href')
        housings_list.append([detail_dict,link])
#     Turn to next page and get more housings
    if soup.find('ul',class_='navnext'):
        if soup.find('ul',class_='navnext').find('a'):
            next_page_token = soup.find('ul',class_='navnext').find('a').get('href')
            next_page = 'https://www.spareroom.co.uk/flatshare/' + next_page_token
        # Use recursion to get all the attractions    
            housings_list.extend(get_housings(next_page))
    return housings_list



In [13]:
url_liver = 'https://www.spareroom.co.uk/flatshare/?search_id=746073457&'
list_liver = get_housings(url_liver)
print(f'We have find {len(list_liver)} results for you.')

We have find 732 results for you.


In [ ]:
list_liver

In [ ]:
def get_details(list_housings):
    count = 0 
    full_dict = dict()
    for dict_info, link in list_housings:
        import requests
        from bs4 import BeautifulSoup
        # Request for details of each housing in NY
        try:
            results_page = requests.get(link)
        except:
            continue
        try:
            soup = BeautifulSoup(results_page.content,'lxml')
        except:
            continue
        # Get housing price for each room 
        price_list = list()
        if soup.find('ul', class_='room-list'): 
            prices = soup.find('ul', class_='room-list')
            price = prices.find_all('strong', class_='room-list__price')
            for price_i in price:
                price_list.append(price_i.get_text())
        # price 这里似乎有个bug，但是我暂时懒得改
        if soup.find('h3', class_='feature__heading'):
            price_list.append(soup.find('h3', class_='feature__heading').get_text())
        # Clean the data to remove 'Availability' in the price list
        if 'Availability' in price_list:
            price_list.remove('Availability')
        dict_info['Price'] = price_list
        # Get Availability for each housing
        avail = soup.find('section', class_='feature feature--availability')
        if avail:
            all_info_avail = avail.find_all('dd', class_='feature-list__value')
            availablility = all_info_avail[0].get_text()      
            min_period = all_info_avail[1].get_text()      
            max_period = all_info_avail[2].get_text() 
            dict_info['Availablility'] = availablility
            dict_info['Min_period'] = min_period
            dict_info['Max_period'] = max_period
        # Get Amenities info for each housing
        amenities = soup.find('section', class_='feature feature--amenities')
        if amenities:
            all_info_amenities = amenities.find_all('dd', class_='feature-list__value')
            if all_info_amenities:
                furnish = all_info_amenities[0].get_text()
                dict_info['Furnishing'] = furnish
        else:
            dict_info['Furnishing'] = 'No info'
        # Female/Male requirement
        preference = soup.find('section', class_='feature feature--household-preferences')
        if preference:
            preference_info = preference.find_all('dt', class_='feature-list__key')
            preference_detail = preference.find_all('dd', class_='feature-list__value')
            if preference_info and preference_detail:
                for info in preference_info:
                    if info.get_text() == 'Gender':
                        info_index = preference_info.index(info)
                        break
                try:
                    gender = preference_detail[info_index].get_text() 
                    dict_info['Gender_requirement'] = gender
                except:
                    dict_info['Gender_requirement'] = 'No info'
        full_dict[count] = dict_info
        count = count + 1
    return full_dict
        

In [ ]:
list_housings_liver = list_liver
dict_liver = get_details(list_housings_liver)

In [ ]:
dict_liver

<h3>Clean the data

In [ ]:
for key in dict_liver.keys():
    for subkey in dict_liver[key].keys():
        if isinstance(dict_liver[key][subkey], str):
            dict_liver[key][subkey] = dict_liver[key][subkey].lstrip().rstrip()
# Clean the price list
for key in dict_liver.keys():
    list_in = list()
    for item in list(dict_liver[key]['Price']):
        list_in.append(item.strip('\n'))
    dict_liver[key]['Price'] = list_in

<h3>Create a dataframe to store the info

In [ ]:
df_info_liver = pd.DataFrame(dict_liver)
df_info_liver = df_info_liver.T

# Fix furnished problem
for i in df_info_liver.index:
    if df_info_liver.iloc[i]['Furnishing'] in ['Furnished', 'Part Furnished', 'Unfurnished']:
        pass
    else:
        df_info_liver.iloc[i]['Furnishing'] = 'No info'
# Fix Gender_requirement problem
for i in df_info_liver.index:
    if not df_info_liver.iloc[i]['Gender_requirement'] in ['Males or females', 'Female preferred', 'No info',
       'Male preferred']:
        df_info_liver.iloc[i]['Gender_requirement'] = 'No info'

In [ ]:
# Create a new col in df_info_liver to store average price for a single person
pattern = r'\d+'  
price_per_month = list()

for i in df_info_liver.index:
    list_in_price = list()
    for item in df_info_liver.iloc[i]['Price']:
        item = item.replace(',', '')
        if 'pw' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info*4)
        if 'pcm' in item:
            price_info = int((re.findall(pattern, item))[0])
            list_in_price.append(price_info)
    price_per_month.append(list_in_price)
price_per_month = np.array(price_per_month)
df_info_liver['Price_per_month'] = price_per_month
list_avg_price = list()
for i in df_info_liver.index:
    list_avg_price.append(np.mean(df_info_liver.iloc[i]['Price_per_month']) )
df_info_liver['Average_price_per_month'] = np.array(list_avg_price)
# Create a new col in df_info_liver to store zip code for each housing




In [ ]:
df_info_liver

In [ ]:
# Create a new col for zip code
list_pos = list()
for i in df_info_liver.index:
    for s in df_info.iloc[i]['Location']:
        if s == '(':
            position =df_info_liver.iloc[i]['Location'].index(s)
            break
    list_pos.append(df_info_liver.iloc[i]['Location'][position:].strip('(').strip(')'))
df_info_liver['Postal_code'] = np.array(list_pos)

<h3>Data Visualization

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')# seaborn is a style that have different colors

<h3>To see if there is a gender preference in Liverpool

In [ ]:
gender_group_liver = df_info_liver.groupby('Gender_requirement')
gender_group_liver.size().plot(kind='bar', title='Gender requirement for rent in Liverpool')

<h3>To see if there is more furnished apartment in Liverpool

In [ ]:
furnish_group_liver = df_info_liver.groupby('Furnishing')
furnish_group_liver.size().plot(kind='bar', title='Furnishing status of housing in Liverpool')

<h3>To see the size distribution in Liverpool 

In [ ]:
size_group_liver = df_info_liver.groupby('Size')
size_group_liver.size().plot(kind='bar', title='Housing size in Liverpool')

<h3>To see the average per-month price change

In [ ]:
df_info_liver['Average_price_per_month'].plot(kind='line', title='Average price in Liverpool')

In [ ]:
# Group the average price
def GroupColFunc(df, ind, col):
    if df[col].loc[ind] < 500:
        return 'Lower than 500'
    elif 1000 > df[col].loc[ind] >= 500:
        return 'Between 500 and 1000'
    else:
        return 'Above 1000'
avg_price_group_liver = df_info_liver.groupby(lambda x: GroupColFunc(df_info, x, 'Average_price_per_month')) 
avg_price_group_liver.size().plot(kind='bar', title='Housing average price in London')
# Add a col of average price group for further analysis
avg_price_list = list()
for i in df_info_liver.index:
    avg_price_list.append(GroupColFunc(df_info, i, 'Average_price_per_month'))
df_info_liver['Avg_price_group'] = np.array(avg_price_list)

In [ ]:
# Group by both size and average price
size_avg_groups_liver = df_info_liver.groupby(['Size','Avg_price_group']).size().unstack()

In [ ]:
COL_NUM = 2
ROW_NUM = 2
import matplotlib.pyplot as plt
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(12,12))

for i, (avg, size) in enumerate(size_avg_groups_liver.items()): 
    ax = axes[int(i/COL_NUM), i%COL_NUM]
    size = size.sort_values(ascending=False) # sort the data in descending order 
    size.plot(kind='barh', ax=ax)
    ax.set_title(avg)

plt.tight_layout() 

<h1> New York

<h1> Manchester